<a href="https://colab.research.google.com/github/krikorantranik/Work/blob/main/MoviesWikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this example, I extract data from a Wikipedia list of the most grossing movies go into each of the links and fetch the text of the movie's article. Then I use BERTopic (which is BERT + HDBScan) to find topics in common which class these films in buckets.

This task as some particularities that made it interesting, first fetching and formatting data from HTML, then cleaning the text to get it ready for the modeling.

In [38]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
pd.options.mode.chained_assignment = None
from io import StringIO
from html.parser import HTMLParser
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltkstop = stopwords.words('english')
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
snow = SnowballStemmer(language='english')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import warnings
import seaborn as sns
!pip install bertopic
from bertopic import BERTopic

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 87.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 117.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.4 MB/s eta 0:00:00
  Preparing metadata (s

Scraping a table from a website (in HTML) is as simple as:

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_best-selling_films_in_the_United_States'
tbls = pd.read_html(url)
tbls[0]

,Title,Video release,All formats,VHS,DVD,Blu-ray,Revenue (est.)
0,The Lion King,"March 3, 1995",48500000,"32,000,000[1]","11,900,000 [2]","3,680,000[3]","$852,000,000[4][2][3]"
1,Finding Nemo,"November 4, 2003",40600000,"520,000[5]","38,800,000[2]","1,320,000[6]","$801,400,000[5][2][6]"
2,Snow White and the Seven Dwarfs,"October 28, 1994",36000000,"28,000,000[7]","5,070,000[8]","2,960,000[3]","$598,000,000[9][8][3]"
3,Aladdin,"October 1, 1993",34630000,"30,000,000[10]","2,820,000[11]","1,810,000[12]","$748,000,000[13][14][12]"
4,Beauty and the Beast,"October 30, 1992",34000000,"22,000,000[15]","7,500,000[16]","4,500,000[3]","$554,000,000[17][16][3]"
5,Titanic,"September 1, 1998",30300000,"25,000,000[18]","4,900,000[19][20]","390,000[21]","$655,000,000[22][23][21]"
6,Toy Story,"October 29, 1996",26650000,"21,000,000[24]","4,920,000[25]","730,000[25]","$401,000,000[17][25]"
7,Avatar,"April 22, 2010",26500000,—,"19,000,000[26]","7,450,000[3]","$400,000,000[27][3]"
8,Jurassic Park,"October 4, 1994",26000000,"24,000,000[7]","1,480,000[28]","520,000[29]","$345,000,000[30][28][29]"
9,Spider-Man,"November 1, 2002",26000000,"6,500,000[31]","19,500,000[31]","16,000[32]","$727,000,000[31][32]"


Now, all the URLs from the page

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_best-selling_films_in_the_United_States'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')

urls = []
for link in soup.find_all('a'):
 row = pd.DataFrame({'Name': [link.get('title')], 'link': [link.get('href')]})
 urls.append(row)
urls = pd.concat(urls, axis=0)
urls

,Name,link
0,None,#bodyContent
0,Visit the main page [z],/wiki/Main_Page
0,Guides to browsing Wikipedia,/wiki/Wikipedia:Contents
0,Articles related to current events,/wiki/Portal:Current_events
0,Visit a randomly selected article [x],/wiki/Special:Random
...,...,...
0,None,https://stats.wikimedia.org/#/en.wikipedia.org
0,None,https://foundation.wikimedia.org/wiki/Special:...
0,None,//en.m.wikipedia.org/w/index.php?title=List_of...
0,None,https://wikimediafoundation.org/


Since I am looking for movies and their revenue, I want only tables that have a column named "Title" and "Revenue" (or "Revenue (est.)").

In [43]:
moviestbl = []
for df in tbls:
 if ('Title' in df.columns) and (('Revenue (est.)' in df.columns) or ('Revenue' in df.columns)):
  if ('Revenue (est.)' in df.columns):
   df = df[['Title','Revenue (est.)']]
   df.columns = ['Title','Revenue']
  else:
   df = df[['Title','Revenue']]
  moviestbl.append(df)
moviestbl = pd.concat(moviestbl, axis=0)
moviestbl

,Title,Revenue
0,The Lion King,"$852,000,000[4][2][3]"
1,Finding Nemo,"$801,400,000[5][2][6]"
2,Snow White and the Seven Dwarfs,"$598,000,000[9][8][3]"
3,Aladdin,"$748,000,000[13][14][12]"
4,Beauty and the Beast,"$554,000,000[17][16][3]"
...,...,...
5,Sing 2,"$10,002,325"
6,No Time to Die,"$18,843,270"
7,The Batman,"$11,368,311"
8,Fantastic Beasts: The Secrets of Dumbledore,"$13,971,276"


I'll clean the revenue column and join the movies and hyperlink tables

In [44]:
moviestbl["clean_revenue"] = moviestbl["Revenue"].astype(str).str.replace('\$','').astype(str).str.replace(' ','').astype(str).str.replace(',','').apply(lambda x: x.split('[')[0])
moviestbl["clean_revenue"] = pd.to_numeric(moviestbl["clean_revenue"], errors='coerce')
moviestbl = pd.merge(left=moviestbl, right=urls, left_on='Title', right_on='Name')
moviestbl = moviestbl[["Title","clean_revenue","link"]]
moviestbl["link"] = np.where(moviestbl["link"].str.contains('https://en.wikipedia.org'),moviestbl["link"],'https://en.wikipedia.org' + moviestbl["link"])
moviestbl = moviestbl.drop_duplicates(ignore_index=True)
moviestbl = moviestbl.sort_values(['Title','clean_revenue'], ascending=False)
moviestbl = moviestbl.groupby('Title').first()
moviestbl = moviestbl.reset_index(drop=False)
moviestbl

,Title,clean_revenue,link
0,Aladdin and the King of Thieves,257000000.0,https://en.wikipedia.org/wiki/Aladdin_and_the_...
1,American Sniper,73823666.0,https://en.wikipedia.org/wiki/American_Sniper
2,Avengers: Age of Ultron,68150184.0,https://en.wikipedia.org/wiki/Avengers:_Age_of...
3,Avengers: Endgame,104154360.0,https://en.wikipedia.org/wiki/Avengers:_Endgame
4,Avengers: Infinity War,112828246.0,https://en.wikipedia.org/wiki/Avengers:_Infini...
...,...,...,...
121,Wedding Crashers,138091895.0,https://en.wikipedia.org/wiki/Wedding_Crashers
122,Wonder Woman 1984,38742601.0,https://en.wikipedia.org/wiki/Wonder_Woman_1984
123,Wreck-It Ralph,112300386.0,https://en.wikipedia.org/wiki/Wreck-It_Ralph
124,X-Men Origins: Wolverine,101074698.0,https://en.wikipedia.org/wiki/X-Men_Origins:_W...


Now it is time to extract the text (which is in the tag paragraph) from each wikipedia page.

In [45]:
def scraptext(txt):
 reqs = requests.get(txt)
 soup = BeautifulSoup(reqs.text, 'html.parser')
 ps = []
 for par in soup.find_all('p'):
  ps.append(par.text)
 ps = ' '.join(ps)
 return ps

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def replace_words(tt, lookp_dict):
 temp = tt.split()
 res = []
 for wrd in temp:
  res.append(lookp_dict.get(wrd, wrd))
 res = ' '.join(res)
 return res

def preprepare(eingang):
 ausgang = strip_tags(eingang)
 ausgang = eingang.lower()
 ausgang = ausgang.replace(u'\xa0', u' ')
 ausgang = re.sub(r'^\s*$',' ',str(ausgang))
 ausgang = ausgang.replace('|', ' ')
 ausgang = ausgang.replace('ï', ' ')
 ausgang = ausgang.replace('»', ' ')
 ausgang = ausgang.replace('¿', '. ')
 ausgang = ausgang.replace('ï»¿', ' ')
 ausgang = ausgang.replace('"', ' ')
 ausgang = ausgang.replace("'", " ")
 ausgang = ausgang.replace('?', ' ')
 ausgang = ausgang.replace('!', ' ')
 ausgang = ausgang.replace(',', ' ')
 ausgang = ausgang.replace(';', ' ')
 ausgang = ausgang.replace('.', ' ')
 ausgang = ausgang.replace("(", " ")
 ausgang = ausgang.replace(")", " ")
 ausgang = ausgang.replace("{", " ")
 ausgang = ausgang.replace("}", " ")
 ausgang = ausgang.replace("[", " ")
 ausgang = ausgang.replace("]", " ")
 ausgang = ausgang.replace("~", " ")
 ausgang = ausgang.replace("@", " ")
 ausgang = ausgang.replace("#", " ")
 ausgang = ausgang.replace("$", " ")
 ausgang = ausgang.replace("%", " ")
 ausgang = ausgang.replace("^", " ")
 ausgang = ausgang.replace("&", " ")
 ausgang = ausgang.replace("*", " ")
 ausgang = ausgang.replace("<", " ")
 ausgang = ausgang.replace(">", " ")
 ausgang = ausgang.replace("/", " ")
 ausgang = ausgang.replace("\\", " ")
 ausgang = ausgang.replace("`", " ")
 ausgang = ausgang.replace("+", " ")
 ausgang = ausgang.replace("=", " ")
 ausgang = ausgang.replace("_", " ")
 ausgang = ausgang.replace("-", " ")
 ausgang = ausgang.replace(':', ' ')
 ausgang = ausgang.replace('\n', ' ').replace('\r', ' ')
 ausgang = ausgang.replace(" +", " ")
 ausgang = ausgang.replace(" +", " ")
 ausgang = ausgang.replace('?', ' ')
 ausgang = re.sub('[^a-zA-Z]', ' ', ausgang)
 ausgang = re.sub(' +', ' ', ausgang)
 ausgang = re.sub('\ +', ' ', ausgang)
 ausgang = re.sub(r'\s([?.!"](?:\s|$))', r'\1', ausgang)
 return ausgang


In [46]:
moviestbl["text"] = moviestbl["link"].apply(lambda x: scraptext(x))
moviestbl["text"] = moviestbl["text"].apply(lambda x: preprepare(x))
moviestbl["text"] = moviestbl["text"].apply(lambda x: ' '.join([word for word in x.split() if word not in (nltkstop)]))
moviestbl = moviestbl[moviestbl["text"] != " "]
moviestbl = moviestbl[moviestbl["text"] != ""]
moviestbl = moviestbl.dropna()

moviestbl

,Title,clean_revenue,link,text
0,Aladdin and the King of Thieves,257000000.0,https://en.wikipedia.org/wiki/Aladdin_and_the_...,aladdin king thieves american direct video ani...
1,American Sniper,73823666.0,https://en.wikipedia.org/wiki/American_Sniper,american sniper american biographical war dram...
2,Avengers: Age of Ultron,68150184.0,https://en.wikipedia.org/wiki/Avengers:_Age_of...,avengers age ultron american superhero film ba...
3,Avengers: Endgame,104154360.0,https://en.wikipedia.org/wiki/Avengers:_Endgame,avengers endgame american superhero film based...
4,Avengers: Infinity War,112828246.0,https://en.wikipedia.org/wiki/Avengers:_Infini...,avengers infinity war american superhero film ...
...,...,...,...,...
121,Wedding Crashers,138091895.0,https://en.wikipedia.org/wiki/Wedding_Crashers,wedding crashers american romantic comedy dire...
122,Wonder Woman 1984,38742601.0,https://en.wikipedia.org/wiki/Wonder_Woman_1984,wonder woman also known ww american superhero ...
123,Wreck-It Ralph,112300386.0,https://en.wikipedia.org/wiki/Wreck-It_Ralph,wreck ralph american computer animated comedy ...
124,X-Men Origins: Wolverine,101074698.0,https://en.wikipedia.org/wiki/X-Men_Origins:_W...,x men origins wolverine american superhero fil...


Now the data is ready for NLP, but I don't think we have enough data to do a model where we train from scratch (Doc2Vec for example), so I will use BERT, in specific Bertopic, which is a package specifically designed for topic modeling (anyways, NLP is not the main demonstration for this exercise)

In [47]:
model = BERTopic(verbose=True,embedding_model='bert-base-uncased', min_topic_size= 3)
headline_topics, _ = model.fit_transform(moviestbl["text"])

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2023-10-11 23:22:08,718 - BERTopic - Transformed documents to Embeddings
2023-10-11 23:22:11,956 - BERTopic - Reduced dimensionality
2023-10-11 23:22:11,967 - BERTopic - Clustered reduced embeddings


As simple as that. Now I assign the topics to the text and explore the topic information.

In [48]:
moviestbl["topic"] = headline_topics
topicinfo = model.get_topic_info()
print(len(topicinfo))
topicinfo

10


,Topic,Count,Name,Representation,Representative_Docs
0,-1,32,-1_film_million_released_shrek,"[film, million, released, shrek, disney, first...",[transformers dark moon american science ficti...
1,0,19,0_film_million_john_released,"[film, million, john, released, fast, highest,...",[american sniper american biographical war dra...
2,1,19,1_film_man_spider_million,"[film, man, spider, million, marvel, would, av...",[spider man far home american superhero film b...
3,2,10,2_film_disney_million_animated,"[film, disney, million, animated, nemo, findin...",[finding nemo american computer animated comed...
4,3,10,3_film_jurassic_world_trevorrow,"[film, jurassic, world, trevorrow, million, di...",[jurassic world american science fiction actio...
5,4,9,4_film_harry_jackson_million,"[film, harry, jackson, million, ring, gandalf,...",[lord rings two towers epic fantasy adventure ...
6,5,9,5_film_million_star_wars,"[film, million, star, wars, force, released, j...",[star wars last jedi also known star wars epis...
7,6,8,6_toy_story_film_pixar,"[toy, story, film, pixar, woody, wall, disney,...",[toy story american computer animated comedy d...
8,7,4,7_million_film_bee_trolls,"[million, film, bee, trolls, barry, weekend, p...",[trolls world tour american computer animated ...
9,8,4,8_bella_twilight_saga_edward,"[bella, twilight, saga, edward, film, part, br...",[twilight saga eclipse american romantic fanta...


Some more useful information about a topic are the words attached to it and their score, and at the same time let's list the movies assigned to it

In [52]:
topic = 0
print("topic: " + str(topic))
print(model.get_topic(topic))
print(moviestbl[moviestbl['topic']==topic]['Title'])
topic = 1
print("topic: " + str(topic))
print(model.get_topic(topic))
print(moviestbl[moviestbl['topic']==topic]['Title'])
topic = 2
print("topic: " + str(topic))
print(model.get_topic(topic))
print(moviestbl[moviestbl['topic']==topic]['Title'])
topic = 3
print("topic: " + str(topic))
print(model.get_topic(topic))
print(moviestbl[moviestbl['topic']==topic]['Title'])
topic = 4
print("topic: " + str(topic))
print(model.get_topic(topic))
print(moviestbl[moviestbl['topic']==topic]['Title'])
topic = 5
print("topic: " + str(topic))
print(model.get_topic(topic))
print(moviestbl[moviestbl['topic']==topic]['Title'])
topic = 6
print("topic: " + str(topic))
print(model.get_topic(topic))
print(moviestbl[moviestbl['topic']==topic]['Title'])
topic = 7
print("topic: " + str(topic))
print(model.get_topic(topic))
print(moviestbl[moviestbl['topic']==topic]['Title'])
topic = 8
print("topic: " + str(topic))
print(model.get_topic(topic))
print(moviestbl[moviestbl['topic']==topic]['Title'])


topic: 0
[('film', 0.04350569277416572), ('million', 0.022194064240271066), ('john', 0.014798302846071271), ('released', 0.01418905828567767), ('fast', 0.014161767983753038), ('highest', 0.01383736239353271), ('grossing', 0.013813122092321889), ('furious', 0.013440486273965685), ('jesus', 0.013194278731592977), ('would', 0.012669821390317413)]
1                         American Sniper
16                       Fast & Furious 6
22                              Furious 7
29                             Home Alone
31         Ice Age: Dawn of the Dinosaurs
32                  Ice Age: The Meltdown
37      John Wick: Chapter 3 – Parabellum
38                Jumanji: The Next Level
50     National Treasure: Book of Secrets
51                    Night at the Museum
60     Sherlock Holmes: A Game of Shadows
65                                Skyfall
76                                Taken 2
79                  The Croods: A New Age
84                           The Hangover
85                   The